# train ([tflm-hello-world](https://github.com/tensorflow/tflite-micro/tree/main/tensorflow/lite/micro/examples/hello_world))


In [ ]:
import math
import os

from absl import app
from absl import flags
from absl import logging
import numpy as np
import tensorflow as tf

def get_data():

  x_values = np.random.uniform(low=0, high=2 * math.pi,size=1000).astype(np.float32)

  np.random.shuffle(x_values)

  y_values = np.sin(x_values).astype(np.float32)

  return (x_values, y_values)

In [ ]:
def create_model() -> tf.keras.Model:
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(1, )))

  model.add(tf.keras.layers.Dense(16, activation='relu'))

  model.add(tf.keras.layers.Dense(1))

  model.compile(optimizer='adam', loss='mse', metrics=['mae'])

  return model

In [ ]:
def convert_tflite_model(model):

  converter = tf.lite.TFLiteConverter.from_keras_model(model)
  tflite_model = converter.convert()
  return tflite_model


In [ ]:
def save_tflite_model(tflite_model, save_dir, model_name):

  if not os.path.exists(save_dir):
    os.makedirs(save_dir)
  save_path = os.path.join(save_dir, model_name)
  with open(save_path, "wb") as f:
    f.write(tflite_model)
  logging.info("Tflite model saved to %s", save_dir)

In [ ]:
def train_model(epochs, x_values, y_values):

  model = create_model()
  model.fit(x_values,
            y_values,
            epochs=epochs,
            validation_split=0.2,
            batch_size=64,
            verbose=2)

  return model

In [ ]:
x_values, y_values = get_data()
trained_model = train_model(100, x_values, y_values)

  # Convert and save the model to .tflite
tflite_model = convert_tflite_model(trained_model)
save_tflite_model(tflite_model,
                  "./",
                    model_name="hello_world_float.tflite")

Epoch 1/100
13/13 - 1s - loss: 1.4937 - mae: 1.0146 - val_loss: 1.2403 - val_mae: 0.9566 - 984ms/epoch - 76ms/step
Epoch 2/100
13/13 - 0s - loss: 0.9198 - mae: 0.8404 - val_loss: 0.7898 - val_mae: 0.7999 - 74ms/epoch - 6ms/step
Epoch 3/100
13/13 - 0s - loss: 0.6321 - mae: 0.7148 - val_loss: 0.5727 - val_mae: 0.6877 - 80ms/epoch - 6ms/step
Epoch 4/100
13/13 - 0s - loss: 0.4899 - mae: 0.6238 - val_loss: 0.4929 - val_mae: 0.6277 - 84ms/epoch - 6ms/step
Epoch 5/100
13/13 - 0s - loss: 0.4340 - mae: 0.5743 - val_loss: 0.4412 - val_mae: 0.5881 - 87ms/epoch - 7ms/step
Epoch 6/100
13/13 - 0s - loss: 0.3893 - mae: 0.5378 - val_loss: 0.3865 - val_mae: 0.5501 - 70ms/epoch - 5ms/step
Epoch 7/100
13/13 - 0s - loss: 0.3398 - mae: 0.5033 - val_loss: 0.3527 - val_mae: 0.5269 - 70ms/epoch - 5ms/step
Epoch 8/100
13/13 - 0s - loss: 0.3110 - mae: 0.4812 - val_loss: 0.3210 - val_mae: 0.5009 - 81ms/epoch - 6ms/step
Epoch 9/100
13/13 - 0s - loss: 0.2840 - mae: 0.4581 - val_loss: 0.2963 - val_mae: 0.4820 - 82m

# hello world lab

In [ ]:
!git clone https://github.com/Yui-Arthur/tinyML_TFLM_lab.git

Cloning into 'tinyML_TFLM_lab'...
remote: Enumerating objects: 697, done.
remote: Counting objects: 100% (697/697), done.
remote: Compressing objects: 100% (450/450), done.
remote: Total 697 (delta 254), reused 674 (delta 231), pack-reused 0
Receiving objects: 100% (697/697), 4.70 MiB | 13.90 MiB/s, done.
Resolving deltas: 100% (254/254), done.


In [ ]:
%cd tinyML_TFLM_lab/hello_world_lab/hello_world

/content/tinyML_TFLM_lab/hello_world_lab/hello_world


## run script

In [ ]:
# setup the tflm lib
# https://github.com/ARM-software/ML-examples/blob/main/tflm-cmsisnn-mbed-image-recognition/setup.sh
!bash setup.sh
# change flatbuffers header to fb_*
# https://github.com/tensorflow/tflite-micro/issues/1226
!bash preprocess.sh

## export .tflite model to .cc

In [ ]:
!xxd -i /content/hello_world_float.tflite > model.cc
!echo -ne "#include \"model_data.h\"\nalignas(8)\n" > model_data.cc
!cat model.cc >> model_data.cc
!sed -i -E 's/(unsigned\s.*\s).*(_len|\[\])/const \1model_data\2/g' model_data.cc

# compile

In [ ]:
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3 << '2'
!sudo apt install python3-pip
!sudo apt-get install python3.7-distutils
!python -m pip install --upgrade --force-reinstall pip

In [ ]:
!python -V
!pip -V

Python 3.7.17
pip 23.3.1 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [ ]:
!sudo apt install git mercurial libusb-1.0-0-dev
!pip install mbed-cli
# install python dependency

!wget https://raw.githubusercontent.com/ARMmbed/mbed-os/master/requirements.txt
!sed -i 's/psutil==5.6.7/ /' requirements.txt
!pip install -r requirements.txt

In [ ]:
!wget -O gcc-arm-none-eabi-9-2020-q2-update-x86_64-linux.tar.bz2 https://developer.arm.com/-/media/Files/downloads/gnu-rm/9-2020q2/gcc-arm-none-eabi-9-2020-q2-update-x86_64-linux.tar.bz2?revision=05382cca-1721-44e1-ae19-1e7c3dc96118&rev=05382cca172144e1ae191e7c3dc96118&hash=FDE675133A099796BD1507A3FF215AC4
!sudo tar -jxvf ./gcc-arm-none-eabi-9-2020-q2-update-x86_64-linux.tar.bz2 -C /usr/local/

In [ ]:
!pip install psutil
!pip install Jinja2
!pip install jsonschema
!pip install Pillow
!pip install numpy

In [ ]:
!mbed config -G GCC_ARM_PATH /usr/local/gcc-arm-none-eabi-9-2020-q2-update/bin
!mbed config -G TOOLCHAIN GCC_ARM
!mbed add mbed-os
!mbed deploy

In [ ]:
!mbed add /content/hello_world/libtensorflow-microlite.a

In [ ]:
!sudo mbed compile